# 실습예제 1-1) Google Colab 실습 환경 안내

# 📚 Google Colab 안내서
Google Colab(Colaboratory)은 **브라우저에서 Python/Jupyter 노트북을 실행**할 수 있는 구글의 무료 클라우드 서비스입니다.  
GPU/TPU를 손쉽게 사용하고, 구글 드라이브와 연동하여 데이터/노트북을 관리할 수 있습니다.

---

## ✅ 핵심 특징

- **클라우드 Jupyter Notebook**: 로컬 설치 없이 바로 실행
- **무료/유료 GPU·TPU 지원**: T4, L4, A100(유료) 등
- **Google Drive 연동**: 파일 저장·공유 간편
- **손쉬운 공유**: URL로 협업(보기/편집 권한 설정)
- **풍부한 예제/라이브러리**: pip로 즉시 설치 가능

---

## 🚀 시작하기

1. 접속: https://colab.research.google.com  
2. `파일 → 새 노트` 또는 `GitHub/Drive`에서 노트 열기  
3. `런타임 → 런타임 유형 변경 → 하드웨어 가속기: GPU/TPU` 선택

---

## 🧰 기본 사용법 & 유용한 코드

### 1) 환경 확인


In [ ]:
import sys, torch, platform
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

## FLOPs/MAC 계산 실습

In [ ]:
# 라이브러리 설치
!pip install calflops

In [ ]:
import torch
from calflops import calculate_flops

def parse_flop_string(s):
    return float(s.strip().split()[0])

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights='ResNet18_Weights.DEFAULT')

flops, macs, params = calculate_flops(
    model=model,
    input_shape=(1, 3, 224, 224),
    print_results=False
)

print(type(flops))
print(params)

print(f"FLOPs: {parse_flop_string(flops):.2f}G, MACs: {parse_flop_string(macs):.2f}G")


# 실습예제 1-2) GPU Utilization vs MFU 개념 및 실습 개요

---

## 🎯 학습 목표
이 실습에서는 **GPU Utilization(활용률)** 과 **MFU(Model FLOPs Utilization)** 의 차이를 이해하고  
직접 실험을 통해 두 지표가 서로 다른 의미를 가지는 이유를 관찰합니다.

---

## 🧠 핵심 개념 비교

| 항목 | GPU Utilization | MFU (Model FLOPs Utilization) |
|------|----------------|------------------------------|
| **정의** | GPU가 현재 얼마나 바쁘게 동작 중인지 (%) | GPU가 낼 수 있는 최대 연산 성능 대비 실제 모델이 활용한 비율 (%) |
| **측정 방법** | `nvidia-smi` 명령어 등으로 실시간 사용률 확인 | FLOPs, 실행 시간, GPU 이론 성능을 기반으로 계산 |
| **단위** | % | % |
| **의미** | GPU가 일하는 시간의 비율 | GPU가 “얼마나 효율적으로” 일했는가 |
| **주요 병목 요인** | 데이터 로딩, I/O, 동기화 지연 | 연산 최적화 부족, 배치 크기, 커널 효율 |
| **활용 목적** | 시스템 상태 확인 | 모델 최적화 및 효율 분석 |

---

## 🧩 실습 구성 단계

1️⃣ **GPU 환경 확인**  
현재 Colab 또는 로컬 환경에서 GPU 사용 가능 여부와 사양 확인

In [21]:
import torch, time, os

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    os.system("nvidia-smi | head -n 20")

CUDA available: True
GPU: NVIDIA A100-SXM4-80GB


⚙️ 2️⃣ 간단한 모델 정의 (Conv + FC)

In [22]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 32, 3, stride=2, padding=1) #   x=(1,3,224,224)
                                                             #   w=(32,3,3,3)
                                                             # out=(1,32,112,112)

                                                             # (N+2P-F)/S+1
                                                             # (224+2-3)/2+1
        self.fc = nn.Linear(32 * 112 * 112, 10) # (1,32*112*112)(32*112*112,10)=>(1,10)
    def forward(self, x):
        x = torch.relu(self.conv(x))
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = SimpleCNN().cuda().eval()

🧮 3️⃣ FLOPs 계산 (모델 연산량)

In [23]:
# FLOPs = 2 * H * W * Cin * Cout * Kh * Kw
H, W, Cin, Cout, Kh, Kw = 224, 224, 3, 32, 3, 3
flops_conv = 2 * H/2 * W/2 * Cin * Cout * Kh * Kw  # stride=2 → H/2,W/2
flops_fc = 2 * (32 * 112 * 112) * 10
total_flops = flops_conv + flops_fc
print(f"총 FLOPs: {total_flops/1e9:.3f} GFLOPs")

총 FLOPs: 0.030 GFLOPs


⏱️ 4️⃣ 실행 시간 측정 (Forward + Backward)

In [24]:
x = torch.randn(32, 3, 224, 224).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
y = torch.randint(0, 10, (32,)).cuda()

torch.cuda.synchronize()
start = time.time()

for _ in range(50):  # 50 iterations
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()

torch.cuda.synchronize()
end = time.time()
train_time = (end - start) / 50
print(f"평균 반복당 학습 시간: {train_time:.4f} 초")

평균 반복당 학습 시간: 0.0193 초


5️⃣ 🧩 MFU 계산식

$$
MFU = \frac{(\text{모델의 총 FLOPs} / \text{실행 시간})}{\text{GPU의 이론적 FLOPs}} \times 100
$$

- $\text{모델의 총 FLOPs}$ : 모델이 한 번의 forward/backward에서 수행한 부동소수점 연산 수  
- $\text{GPU의 이론적 FLOPs}$ : GPU가 낼 수 있는 최대 부동소수점 연산량 (A100 기준 $19.5 \times 10^{12}$ FLOPs/s)  
- **결과 단위:** %

In [25]:
gpu_theoretical_flops = 19.5e12  # A100 기준 (FP32)
mfu = (total_flops / train_time) / gpu_theoretical_flops * 100
print(f"🔹 MFU(Model FLOPs Utilization): {mfu:.2f}%")

🔹 MFU(Model FLOPs Utilization): 0.01%


🔎 6️⃣ GPU Utilization 확인

아래 셀을 실행해 실시간 GPU 활용률을 확인합니다.
(Colab에서는 watch 명령이 지원되지 않으므로 단발성 출력으로 확인합니다.)

In [26]:
!nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.used --format=csv

utilization.gpu [%], utilization.memory [%], memory.used [MiB]
0 %, 0 %, 827 MiB


📈 7️⃣ 결과 비교

| 지표                  | 의미                  | 계산 기준            | 예시 값    |
| ------------------- | ------------------- | ---------------- | ------- |
| **GPU Utilization** | GPU가 바쁘게 일한 비율      | 실시간 GPU 활용률(%)   | 예: 85 % |
| **MFU**             | GPU의 이론 연산 대비 실제 효율 | FLOPs / 이론 FLOPs | 예: 25 % |

➡️ GPU 활용률은 높더라도, 실제 연산 효율(MFU)은 낮을 수 있습니다.
이는 데이터 로딩 지연, 메모리 대역폭 한계, 작은 배치 크기 등의 요인 때문입니다.


## 📘 기대 학습 효과

- GPU 활용률과 실제 연산 효율의 차이를 정량적으로 이해  
- 모델 구조, 배치 크기, 최적화 기법이 효율에 미치는 영향 인식  
- MFU 계산을 통해 병목 구간을 찾아 성능 최적화 방향 제시 가능


# 🧮 FLOPs (Floating Point Operations) 개념 정리

---

## 📘 정의
**FLOPs(Floating Point Operations)**란  
모델이 수행하는 **부동소수점 연산의 총 개수**를 의미합니다.  

즉, 신경망이 학습 또는 추론 과정에서  
곱셈(`×`), 덧셈(`+`), 나눗셈(`/`), 지수(`exp`) 등의 **실수 연산이 몇 번 수행되었는가**를 정량적으로 나타내는 지표입니다.

---

## 🧠 왜 FLOPs가 중요한가?

| 항목 | 설명 |
|------|------|
| **계산 복잡도 지표** | 모델의 연산량을 정량화하여 복잡도를 비교할 수 있음 |
| **성능·속도 예측** | FLOPs가 많을수록 GPU 연산량과 실행 시간이 증가함 |
| **효율성 판단 기준** | FLOPs가 적을수록 계산 효율이 높고, 경량 모델에 유리함 |
| **하드웨어 비교 기준** | 서로 다른 GPU/TPU 환경 간 모델 효율을 정량 비교 가능 |

---


# 💾 부동소수점 표준: IEEE 754 요약

---

## 1) 개요
**IEEE 754**는 컴퓨터에서 실수를 표현·연산하는 국제 표준입니다.  
핵심 아이디어는 실수를 **부호(Sign)·지수(Exponent)·가수(Fraction)** 로 나눠 **정규화된 과학적 표기**로 저장한다는 것.

실수 값은 (정규수의 경우) 다음으로 해석됩니다:
$$
(-1)^s \times (1.f)_2 \times 2^{\,e - \text{bias}}
$$
- $s$: 부호 비트(0=양수, 1=음수)  
- $f$: 가수부(숨겨진 1 포함 전제)  
- $e$: 지수부 정수값  
- $\text{bias}$: 형식별 바이어스

---

## 2) 대표 포맷 (Binary)
| 포맷 | 총 비트 | 부호 | 지수 | 가수 | 바이어스 |
|---|---:|---:|---:|---:|---:|
| **binary32 (float)** | 32 | 1 | 8 | 23 | 127 |
| **binary64 (double)** | 64 | 1 | 11 | 52 | 1023 |

> 정밀도(유효 십진자리 근사): float ≈ 7자리, double ≈ 15~16자리

---

## 3) 특수 값 인코딩
지수와 가수의 비트 패턴 조합으로 특수 값이 정의됩니다.

- **정규수(normal)**: $0 < e < \text{max}$  
  값: $(-1)^s (1.f) 2^{e-\text{bias}}$
- **서브노말(subnormal)**: $e=0$, $f\neq0$  
  값: $(-1)^s (0.f) 2^{1-\text{bias}}$ → **아주 작은 수의 연속성 보장**
- **0**: $e=0$, $f=0$ → `+0`, `-0` 존재 (부호만 다름)
- **∞ (무한대)**: $e=\text{max}$, $f=0$ → `+∞`, `-∞`
- **NaN**: $e=\text{max}$, $f\neq0$  
  - **qNaN**(quiet), **sNaN**(signaling) 구분 — 계산 전파/예외 신호

---

## 4) 반올림 모드
기본은 **ties-to-even(가장 가까운 값, 동점은 짝수로)**. 그 외:
- toward $+\infty$, toward $-\infty$, toward $0$, ties-away-from-zero (확장)

반올림은 **연산, 변환, 저장** 단계에서 적용되어 결과 오차를 결정합니다.

---

## 5) 핵심 지표
- **기계 엡실론 $\epsilon$**: $1$과 구분되는 가장 작은 값 차이  
  - float: $\epsilon \approx 2^{-23} \approx 1.19\times10^{-7}$  
  - double: $\epsilon \approx 2^{-52} \approx 2.22\times10^{-16}$
- **ULP (Unit in the Last Place)**: 인접 표현가능 수 간 간격

---

## 6) 자주 겪는 현상과 주의점
- **이진 표현 불가능**: $0.1_{10}$, $0.2_{10}$ 등은 이진에서 **무한소수** → 근사 저장  
  ⇒ `0.1 + 0.2 != 0.3`
- **연산 비결합성**: $(a+b)+c \neq a+(b+c)$ (반올림 순서 영향)  
  ⇒ 누적합은 Kahan/Neumaier 보정 합계 기법 고려
- **소실/취소(cancellation)**: 비슷한 큰 수의 차 $a-b$에서 유효자리 손실  
  ⇒ **안정한 수식 변형** 필요
- **서브노말 성능**: 매우 작은 수 영역에서 느려질 수 있음 (denormals-are-zero 옵션 등 하드웨어 종속)

---

## 7) 예시 (binary32)
- `1.0f` → `0x3F800000`  
- `-0.0f` → `0x80000000`  
- `+∞` → `0x7F800000`, `NaN` → `0x7FC00000`(예)

---

## 8) 확장 포맷과 십진 포맷
- **binary128**(quadruple, 128비트), **bfloat16**(지수 8/가수 7) 등 하드웨어·ML 특화 포맷
- **decimal32/64/128**: 십진 반올림/회계 용도, 10진 정밀 보존

---

## 9) 실무 팁
- 비교는 **허용 오차**로: `abs(a-b) <= rtol*max(|a|,|b|) + atol`
- 누적합/내적은 **Kahan 합계**, 고정 순서 reduce, 혹은 **higher precision** 사용
- I/O는 **형식 지정자**로 자릿수 명시(예: `"{:.17g}".format(x)` for double)

---

## 10) 값 해석 공식 요약
- 정규수: $(-1)^s (1.f) 2^{e-\text{bias}}$
- 서브노말: $(-1)^s (0.f) 2^{1-\text{bias}}$

> 이 두 식만 기억하면 어떤 비트 패턴도 실수값으로 해석할 수 있습니다.


# 🧮 FC (Fully Connected) 연산에서 FLOPs 계산 원리

---

## 🎯 개념 요약

완전연결(FC, Fully Connected) 레이어는  
입력 뉴런($N_{in}$)과 출력 뉴런($N_{out}$)이 **모두 연결**된 형태입니다.

각 출력 뉴런은 모든 입력 값에 대해 **가중치 곱셈(Multiply)** 과 **누산(Add)** 을 수행하므로  
연산량은 다음과 같습니다:

$$
FLOPs = 2 \times N_{in} \times N_{out}
$$

---

## 📘 왜 2를 곱할까?

- **1개의 연결선(Weight)** 은 `입력 × 가중치` → **곱셈 연산(1 FLOP)**  
- 여러 입력의 곱셈 결과를 모두 더함 → **덧셈 연산(1 FLOP)**  

즉,  
하나의 weight 연결당 “곱셈 + 덧셈” = **2 FLOPs**

이 때문에 완전연결층 전체의 FLOPs는  
입력과 출력 뉴런의 개수를 곱하고 ×2를 해줍니다.

---

## 🧠 예시

예를 들어 다음과 같은 레이어가 있다고 가정합니다.

- 입력 뉴런 수: $N_{in} = 4$  
- 출력 뉴런 수: $N_{out} = 3$

연산량은 다음과 같습니다.

$$
FLOPs = 2 \times 4 \times 3 = 24
$$

즉,  
**곱셈 12번 + 덧셈 12번 = 총 24 FLOPs**

---

## ⚙️ 실습 코드 예제




In [27]:
import torch
import torch.nn as nn

# FC (Linear) 레이어 정의
N_in, N_out = 4, 3
fc = nn.Linear(N_in, N_out, bias=False)

# 입력 데이터 (배치 크기 1)
x = torch.randn(1, N_in)
y = fc(x)

print("입력 크기:", x.shape)
print("출력 크기:", y.shape)

# FLOPs 계산
flops = 2 * N_in * N_out
print(f"이론적 FLOPs: {flops} 회 연산 (곱셈 + 덧셈 포함)")

입력 크기: torch.Size([1, 4])
출력 크기: torch.Size([1, 3])
이론적 FLOPs: 24 회 연산 (곱셈 + 덧셈 포함)


# 🧮 컨볼루션 연산에서 FLOPs 계산의 의미

---

## 🎯 개념 요약
컨볼루션(Convolution)은 **곱셈(Multiply)** 과 **덧셈(Add)** 연산으로 구성된 대표적인 계산 집약적 연산입니다.

한 번의 컨볼루션 연산에서 **하나의 출력 픽셀**을 얻기 위해 수행되는 연산은 다음과 같습니다:

$$
\text{연산 수} = K_h \times K_w \times C_{in}
$$

즉,  
- 커널(필터) 크기 $K_h \times K_w$ 만큼의 영역을 입력에서 잘라오고,  
- 각 위치마다 $C_{in}$ 개의 채널에 대해 곱셈을 수행하고,  
- 그 결과를 모두 더해($+$) 최종 출력을 구합니다.

곱셈(Multiply)과 덧셈(Add)은 각각 **1 FLOP**으로 계산되므로,  
총 **2 FLOPs**가 한 쌍의 곱셈-덧셈 연산에 해당합니다.

---

## 🧩 FLOPs 계산 공식

$$
FLOPs = 2 \times H_{out} \times W_{out} \times K_h \times K_w \times C_{in} \times C_{out}
$$

| 항목 | 의미 |
|------|------|
| $H_{out}, W_{out}$ | 출력 Feature Map의 높이, 너비 |
| $K_h, K_w$ | 커널(필터)의 높이, 너비 |
| $C_{in}$ | 입력 채널 수 |
| $C_{out}$ | 출력 채널(필터 개수) 수 |
| 2배 | Multiply + Add (곱셈 + 덧셈) 연산을 모두 포함하기 때문 |

---

## 🧠 직관적 예시

예를 들어,  
- 입력 크기: $(C_{in}, H, W) = (3, 4, 4)$  
- 커널 크기: $(K_h, K_w) = (3, 3)$  
- 출력 채널 수: $C_{out} = 1$  
- 출력 크기: $(H_{out}, W_{out}) = (2, 2)$  

이라면,

$$
FLOPs = 2 \times 2 \times 2 \times 3 \times 3 \times 3 \times 1 = 216
$$

즉, **216개의 부동소수점 연산**이 1장의 출력 Feature Map을 계산하는 데 사용됩니다.

---

## ⚙️ 실습 코드 예제




In [28]:
import torch
import torch.nn as nn
import numpy as np

# 입력 및 커널 정의
x = torch.randn(1, 3, 4, 4)   # (배치, 채널, 높이, 너비)
conv = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0)

# 연산 수행
y = conv(x)
print("출력 크기:", y.shape) # (1,1,2,2)

# FLOPs 계산 공식 적용
H_out, W_out = y.shape[2], y.shape[3]
K_h, K_w = conv.kernel_size
C_in, C_out = conv.in_channels, conv.out_channels

flops = 2 * H_out * W_out * K_h * K_w * C_in * C_out
print(f"이론적 FLOPs: {flops} 회 연산")

출력 크기: torch.Size([1, 1, 2, 2])
이론적 FLOPs: 216 회 연산


# ⚙️ MACs (Multiply–Accumulate Operations) 개념 정리

---

## 📘 정의
**MACs (Multiply–Accumulate Operations)**는  
딥러닝 연산에서 자주 등장하는 **곱셈(Multiply)** 과 **덧셈(Accumulate)** 연산의 조합을 의미합니다.

즉, 하나의 MAC 연산은 다음 수식을 계산합니다:

$$
y = (a \times b) + c
$$

따라서 **1 MAC = 1 곱셈 + 1 덧셈**  
즉, **1 MAC ≈ 2 FLOPs** 로 환산할 수 있습니다.

---

## 🧠 왜 MACs가 중요한가?

| 항목 | 설명 |
|------|------|
| **연산 효율 지표** | 하드웨어(특히 GPU/TPU)의 실제 연산 단위와 일치 |
| **성능 측정 단위** | CNN, RNN, Transformer 모델의 연산량을 평가할 때 주로 사용 |
| **모델 최적화 기준** | MACs 감소는 FLOPs 감소와 거의 동일하며, **속도·전력 효율** 향상으로 직결 |
| **FPGA·ASIC 설계 기준** | 대부분의 AI 칩은 MAC 기반으로 병렬 연산 구조를 설계함 |

---

## 🧩 수학적 관계

$$
1\ \text{MAC} = 2\ \text{FLOPs}
$$

즉, FLOPs가 $10^9$ (1 GFLOP)이라면  
MACs는 $0.5 \times 10^9$ (0.5 GMAC) 입니다.

---

## 🧮 컨볼루션 연산 예시

컨볼루션 연산은 입력 특징 맵과 필터(커널)를 곱하고 누적합을 계산합니다.

$$
\text{MACs} = H_{out} \times W_{out} \times K_h \times K_w \times C_{in} \times C_{out}
$$

| 기호 | 의미 |
|------|------|
| $H_{out}, W_{out}$ | 출력 Feature Map 크기 |
| $K_h, K_w$ | 커널(필터) 크기 |
| $C_{in}$ | 입력 채널 수 |
| $C_{out}$ | 출력 채널 수 |

> 컨볼루션 한 번당 **각 출력 픽셀**을 계산하기 위해  
> 모든 입력 채널의 **곱셈-누산(MAC)** 연산이 수행됩니다.

---

## 🧩 Fully Connected 연산 예시

완전연결층(FC, Linear Layer)의 경우:
$$
\text{MACs} = N_{in} \times N_{out}
$$

즉, 입력 뉴런($N_{in}$)과 출력 뉴런($N_{out}$)이 모두 연결되어 있다면,  
각 연결마다 1 MAC 연산이 수행됩니다.

> 예: 입력 4096, 출력 1000 → $4096 \times 1000 = 4.1M$ MACs

---

## ⚙️ PyTorch 예제




In [29]:
import torch
import torch.nn as nn

# 간단한 모델 정의
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 32, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(32 * 32 * 32, 10)
)

x = torch.randn(1, 3, 32, 32)

# MACs 계산 예시
H_out, W_out = 32, 32
K_h, K_w = 3, 3
C_in, C_out = 3, 16

conv1_macs = H_out * W_out * K_h * K_w * C_in * C_out
print(f"Conv1 MACs: {conv1_macs / 1e6:.2f} MMACs")

Conv1 MACs: 0.44 MMACs


# 🔢 1 MAC = 2 FLOPs 의 의미

---

## 🎯 핵심 요약
딥러닝이나 수치연산에서  
**1 MAC(Multiply–Accumulate)** 연산은 다음 두 단계를 수행합니다.

1️⃣ **곱셈(Multiply)**  
2️⃣ **덧셈(Add or Accumulate)**

즉, 하나의 MAC 연산은 다음 수식에 해당합니다:

$$
y = (a \times b) + c
$$

이 과정에서:
- `a × b` → **1개의 곱셈 연산 (1 FLOP)**  
- `(a × b) + c` → **1개의 덧셈 연산 (1 FLOP)**  

따라서 전체적으로는  
**1 MAC = 1 Multiply + 1 Add = 2 Floating Point Operations**

---

## 🧮 수학적 관점
$$
1\ \text{MAC} = 2\ \text{FLOPs}
$$

| 연산 종류 | FLOPs 수 | 설명 |
|------------|-----------|------|
| 곱셈(Multiply) | 1 | 실수 × 실수 |
| 덧셈(Add) | 1 | 곱셈 결과를 누산기(accumulator)에 더함 |
| **합계** | **2 FLOPs** | 1 MAC 수행당 2개의 부동소수점 연산 |

---

## 💡 예시로 이해하기

예를 들어,  
다음 계산을 수행한다고 가정합시다:

$$
y = (2.0 \times 3.0) + 1.0
$$

| 단계 | 연산 | FLOPs |
|------|------|--------|
| ① | 2.0 × 3.0 | 1 FLOP |
| ② | (결과 6.0) + 1.0 | 1 FLOP |
| **총합** | **1 MAC = 2 FLOPs** | |

즉, “곱하고 더하는” 1회 연산 = **2개의 부동소수점 연산**으로 구성됩니다.

---

## ⚙️ 하드웨어적 의미
GPU, CPU, TPU 등 대부분의 하드웨어는  
**MAC(Multiply–Accumulate)** 연산 단위를 기반으로 병렬 연산을 수행합니다.

- 하나의 **MAC 유닛(MAC Unit)** 은 두 수를 곱하고 결과를 누적기(accumulator)에 더함  
- 이는 신경망에서 **가중치 × 입력 + 편향** 형태로 자주 사용됨

> 💬 예: CNN, FC, RNN, Transformer 모두 내부적으로 “MAC 연산”으로 구성됨

---

## 📊 FLOPs vs MACs 비교 요약

| 구분 | 의미 | 단위 예시 | 관계 |
|------|------|------------|------|
| **MACs** | 실제 하드웨어의 연산 조합 단위 (Multiply + Add) | GMAC (10⁹ MACs) | — |
| **FLOPs** | 부동소수점 연산 횟수 (곱셈 또는 덧셈) | GFLOPs (10⁹ FLOPs) | 1 MAC = 2 FLOPs |

---

## 🧠 결론

- **1 MAC = 2 FLOPs** 는  
  한 번의 “곱셈 + 덧셈” 조합이 **2개의 부동소수점 연산**을 포함한다는 의미입니다.  
- FLOPs는 연산량을 세밀하게 계산할 때 사용되고,  
  MACs는 하드웨어 효율(실제 처리 단위)을 표현할 때 사용됩니다.

---

✅ **요약 문장**
> 하나의 MAC 연산은 부동소수점 곱셈 1회와 덧셈 1회로 이루어지며,  
> 따라서 **1 MAC = 2 FLOPs** 로 표현됩니다.


# 실습예제 1-3) MFU 측정 도구 사용법

# 🔍 PyTorch Profiler 안내서

`PyTorch Profiler`는 **모델 학습·추론 중 CPU/GPU 연산, 커널, 메모리, I/O** 등을 정밀하게 측정해
**병목(bottleneck)** 을 찾아내는 도구입니다. 타임라인과 통계 테이블을 제공하며
**TensorBoard**와 연동해 시각화할 수 있습니다.

---

## ✅ 무엇을 측정하나요?
- **연산 시간**: op별/레이어별 `self_cpu_time_total`, `self_cuda_time_total`
- **호출 횟수 & 입력 shape**: `count`, `record_shapes`
- **CUDA 커널/스트림 타임라인**: launch 간격, 동기화 지연
- **메모리/파라미터(옵션)**: `profile_memory=True`
- **데이터 로딩**: `DataLoader::get_batch`, `aten::copy_` 등 I/O 비용

---

In [52]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

# ---------------------------------------------------
# 1️⃣ 모델 및 입력 정의
# ---------------------------------------------------
model = torch.nn.Linear(1024, 1024).cuda()
input_data = torch.randn(16, 1024).cuda()

# ---------------------------------------------------
# 2️⃣ FLOPs 계산 (이론값)
# ---------------------------------------------------
batch_size = 16
in_features = 1024
out_features = 1024
flops_per_forward = 2 * batch_size * in_features * out_features  # multiply + add
print(f"[INFO] Estimated FLOPs per forward: {flops_per_forward/1e6:.2f} MFLOPs")

# ---------------------------------------------------
# 3️⃣ FLOPs 측정 (Profiler)
# ---------------------------------------------------
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
             with_flops=True, record_shapes=True) as prof:
    with record_function("linear_forward"):
        output = model(input_data)

total_flops = sum([evt.flops for evt in prof.key_averages() if hasattr(evt, "flops") and evt.flops is not None])
print(f"[INFO] FLOPs measured by profiler: {total_flops/1e6:.2f} MFLOPs")

# ---------------------------------------------------
# 4️⃣ CUDA 실행 시간 측정 (정확한 방식)
# ---------------------------------------------------
torch.cuda.synchronize()
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# 반복 횟수 (짧은 연산 보정용)
num_iter = 10000

start_event.record()
for _ in range(num_iter):
    _ = model(input_data)
end_event.record()

torch.cuda.synchronize()
elapsed_time_ms = start_event.elapsed_time(end_event)  # ms 단위
avg_time_s = (elapsed_time_ms / num_iter) / 1000.0

print(f"[INFO] Avg forward time per iteration: {avg_time_s*1e3:.4f} ms")

# ---------------------------------------------------
# 5️⃣ Throughput & MFU 계산
# ---------------------------------------------------
throughput_gflops = (flops_per_forward / avg_time_s) / 1e9  # GFLOPs/s
gpu_peak_flops = 312_000  # A100 FP16 기준 (GFLOPs)
mfu = (throughput_gflops / gpu_peak_flops) * 100

print(f"[RESULT] Throughput: {throughput_gflops:.2f} GFLOPs/s")
print(f"[RESULT] MFU (Model FLOPs Utilization): {mfu:.4f}%")


[INFO] Estimated FLOPs per forward: 33.55 MFLOPs
[INFO] FLOPs measured by profiler: 33.55 MFLOPs
[INFO] Avg forward time per iteration: 0.0407 ms
[RESULT] Throughput: 823.48 GFLOPs/s
[RESULT] MFU (Model FLOPs Utilization): 0.2639%
